<a href="https://colab.research.google.com/github/MakeItZone/CommunitySensorsDataScience/blob/main/SensorCommunityExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Code to Read Sensor Data from Sensor.Community

Roughly documenting my process of experimentation and discovery.

First, try and find some background info on the task at hand.

- https://github.com/bessarabov/api.sensor.community
- https://github.com/opendata-stuttgart/meta/wiki/EN-APIs

So a REST API.

Not much info on what the expected format will be; let's take a look at the server side code- the code that will recieve and process the data:

- https://github.com/opendata-stuttgart/feinstaub-api/tree/master/feinstaub/sensors

Aside: Django uses the Python code structure to figure out and create database tables.

So even though I don't fully understand the code, I can see what the allowed sensor type names are, etc.

When experimenting, I don't want to pollute the production data- so are there testing facilities?

-https://forum.sensor.community/t/test-phase-settings/739- push to own API to not pollute main database

And how often should data be updated?

- https://forum.sensor.community/t/how-often-should-i-send-data-to-https-api-sensor-community-v1-push-sensor-data/785

What about deleting or changing (test) sensors?

- https://forum.sensor.community/t/reregister-unregister-sensor/494
- https://devices.sensor.community/
-https://sensor.community/en/sensors/airrohr/

it looks like the definition of a node (e.g. addding a sensor, changing the types of sensors, etc) requires an email to support.

During the experimentation phase, we may want to generate ids ourselves so that sensor design can be iterated.

Also, looks like there is a requirement that updates happen at least every 5 minutes for them to be shown on the main map.

Does python have any easy ways to deal with making REST requests?
- https://www.nylas.com/blog/use-python-requests-module-rest-apis/
- https://realpython.com/api-integration-in-python/

Time to start experimenting...

In [1]:
import requests


In [2]:
sensor_id= 70249 # found a sensor from the map
endpoint = f"https://data.sensor.community/airrohr/v1/sensor/{sensor_id}/"
print(f"Endpoint is: {endpoint}")

Endpoint is: https://data.sensor.community/airrohr/v1/sensor/70249/


Check: copy the generated URL above and paste it in a new browser tab. Does it work as expected?

In [6]:
response = requests.get(endpoint)
print(response)
print(response.text) 
print(response.json)

<Response [200]>
[{"sampling_rate":null,"sensordatavalues":[{"value_type":"P2","value":"4","id":28580551259},{"value_type":"P1","value":"5","id":28580551279}],"timestamp":"2022-11-02 21:00:04","sensor":{"pin":"1","sensor_type":{"manufacturer":"Plantower","id":23,"name":"PMS5003"},"id":70249},"location":{"indoor":0,"altitude":"15.0","country":"CA","latitude":"48.416","exact_location":0,"id":57644,"longitude":"-123.374"},"id":12797406423}]
<bound method Response.json of <Response [200]>>


`response.json` is outputting that it's a bound method; ie not a string, but a method we should call.

In [7]:
response.json()

[{'sampling_rate': None,
  'sensordatavalues': [{'value_type': 'P2', 'value': '4', 'id': 28580551259},
   {'value_type': 'P1', 'value': '5', 'id': 28580551279}],
  'timestamp': '2022-11-02 21:00:04',
  'sensor': {'pin': '1',
   'sensor_type': {'manufacturer': 'Plantower', 'id': 23, 'name': 'PMS5003'},
   'id': 70249},
  'location': {'indoor': 0,
   'altitude': '15.0',
   'country': 'CA',
   'latitude': '48.416',
   'exact_location': 0,
   'id': 57644,
   'longitude': '-123.374'},
  'id': 12797406423}]

Interesting; looks like the last two sets of samples. Let's explore a bit more...


In [8]:
json= response.json()
print(json[0])

{'sampling_rate': None, 'sensordatavalues': [{'value_type': 'P2', 'value': '4', 'id': 28580551259}, {'value_type': 'P1', 'value': '5', 'id': 28580551279}], 'timestamp': '2022-11-02 21:00:04', 'sensor': {'pin': '1', 'sensor_type': {'manufacturer': 'Plantower', 'id': 23, 'name': 'PMS5003'}, 'id': 70249}, 'location': {'indoor': 0, 'altitude': '15.0', 'country': 'CA', 'latitude': '48.416', 'exact_location': 0, 'id': 57644, 'longitude': '-123.374'}, 'id': 12797406423}


Ok, that's starting to make sense, let's explore a bit further...


In [9]:
data = json[0]["sensordatavalues"]

In [10]:
data

[{'value_type': 'P2', 'value': '4', 'id': 28580551259},
 {'value_type': 'P1', 'value': '5', 'id': 28580551279}]

In [11]:
sensor=json[0]["sensor"]
sensor

{'pin': '1',
 'sensor_type': {'manufacturer': 'Plantower', 'id': 23, 'name': 'PMS5003'},
 'id': 70249}

In [12]:
location=json[0]["location"]
location

{'indoor': 0,
 'altitude': '15.0',
 'country': 'CA',
 'latitude': '48.416',
 'exact_location': 0,
 'id': 57644,
 'longitude': '-123.374'}

So the data structure looks to be pretty straight forward.

Open question: what generates the `ID`s? 

With luck, only the top level sensor `id` (the one used in the URL **not** the one in the JSON response) is needed for a post.


# POST

TODO

* sign up on devices.community.sensor
* read through the POST documentation and see how to register a new sensor
* set it to report own data (not record on main map)
* push data
* verify it's sent